# 5. LangChain Expression Language（LCEL）徹底解説


In [84]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [85]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 langchain-community==0.3.0

## 5.1. Runnable と RunnableSequence―LCEL の最も基本的な構成要素


In [86]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [87]:
prompt_value = prompt.invoke({"dish": "カレー"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(output)

カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、蓋をして中火にし、約15分煮ます。

5. **カレールーを加える**:
   - 火を止めてカレールーを加え、よく溶かします。再び弱火にし、10分ほど煮込みます。お好みでガーリックパウダーや生姜を加えて風味を調整します。

6. **味を調える**:
   - 最後に塩で味を調整し、全体をよく混ぜます。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- ほうれん草のソテー

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて自分好みのカレーを楽しんでください！


In [88]:
chain = prompt | model | output_parser

In [89]:
output = chain.invoke({"dish": "カレー"})
print(output)

カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら味を見て、必要に応じて塩や胡椒で調整します。

6. **仕上げ**:
   - お好みでガーリックパウダーや生姜を加えて風味をアップさせます。火を止めて、少し冷ますと味がなじみます。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- ほうれん草のソテー

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて自分好みのカレーを楽しんでください！


### Runnable の実行方法―invoke・stream・batch


In [90]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "カレー"}):
    print(chunk, end="", flush=True)

カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら火を止めます。

6. **味を調える**:
   - お好みで塩や胡椒で味を調整します。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- ほうれん草のソテー

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて楽しんでください！

In [91]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "カレー"}, {"dish": "うどん"}])
print(outputs)

['カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。\n\n### 材料（4人分）\n- 鶏もも肉：400g（食べやすい大きさにカット）\n- 玉ねぎ：2個（みじん切り）\n- にんじん：1本（薄切り）\n- じゃがいも：2個（角切り）\n- カレールー：1箱（約200g）\n- サラダ油：大さじ2\n- 水：600ml\n- 塩：適量\n- 胡椒：適量\n- お好みで：ガーリック、しょうが（みじん切り）、トマト（角切り）、ピーマンなど\n\n### 作り方\n1. **下ごしらえ**：\n   - 鶏もも肉に塩と胡椒をふり、下味をつけておきます。\n   - 玉ねぎ、にんじん、じゃがいもをそれぞれ切ります。\n\n2. **炒める**：\n   - 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを入れて中火で炒めます。玉ねぎが透明になるまで炒めます。\n   - にんじんとじゃがいもを加え、さらに炒めます。\n\n3. **鶏肉を加える**：\n   - 鶏もも肉を鍋に加え、表面が白くなるまで炒めます。\n\n4. **煮る**：\n   - 水を加え、沸騰したらアクを取り除きます。蓋をして中火で約15分煮ます。\n\n5. **カレールーを加える**：\n   - カレールーを割り入れ、よく混ぜて溶かします。さらに10分ほど煮込み、全体がなじんだら火を止めます。\n\n6. **味を調える**：\n   - 最後に味を見て、必要であれば塩や胡椒で調整します。\n\n7. **盛り付け**：\n   - ご飯と一緒に盛り付け、お好みでパセリや福神漬けを添えて完成です！\n\n### おすすめのトッピング\n- 煮卵\n- チーズ\n- ナンやライス\n\nこのレシピを参考に、ぜひ美味しいカレーを作ってみてください！お好みで具材をアレンジしても楽しめますよ。', 'うどんのレシピをご紹介します。シンプルで美味しい「かけうどん」の作り方です。\n\n### 材料（2人分）\n- うどん（乾燥または生）: 2玉\n- だし汁: 600ml（昆布と鰹節で取ったものがベストですが、だしの素でも可）\n- 醤油: 大さじ2\n- みりん: 大さじ1\n- 塩: 少々\n- トッピング（お好みで）:\n  - ネギ（小口切り）\n  - 天かす\n  

### LCEL の「|」で様々な Runnable を連鎖させる


In [92]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [93]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [94]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください。"),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [95]:
cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print(output)

10 + 2 * 3 の答えは **16** です。


## 5.2. RunnableLambda―任意の関数を Runnable にする


In [96]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [97]:
from langchain_core.runnables import RunnableLambda


def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | RunnableLambda(upper)

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### chain デコレーターを使った RunnableLambda の実装


In [98]:
from langchain_core.runnables import chain


@chain
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### RunnableLambda への自動変換


In [99]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

In [100]:
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### Runnable の入力の型と出力の型に注意


In [101]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | upper

# 以下のコードを実行するとエラーになります
# output = chain.invoke({"input": "Hello!"})

In [102]:
chain = prompt | model | StrOutputParser() | upper

In [103]:
output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### （コラム）独自の関数を stream に対応させたい場合


In [104]:
from typing import Iterator


def upper(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()


chain = prompt | model | StrOutputParser() | upper

for chunk in chain.stream({"input": "Hello!"}):
    print(chunk, end="", flush=True)

HELLO! HOW CAN I ASSIST YOU TODAY?

## 5.3. RunnableParallel―複数の Runnable を並列で処理する


In [105]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

In [106]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

In [107]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [83]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "生成AIの進化について"})
pprint.pprint(output)

{'optimistic_opinion': '生成AIの進化は本当に素晴らしいですね！技術が進むことで、私たちの生活がより便利で豊かになる可能性が広がっています。クリエイティブな作業や問題解決の手助けをしてくれるAIが増えてきて、私たちのアイデアや夢を実現するためのパートナーとして活躍しています。\n'
                       '\n'
                       'さらに、生成AIは教育や医療、エンターテインメントなど、さまざまな分野での革新を促進しています。これにより、より多くの人々が新しい知識や体験にアクセスできるようになり、社会全体が進化していくのです。未来には、AIと人間が協力して、より良い世界を築いていく姿が見られることでしょう！',
 'pessimistic_opinion': '生成AIの進化は確かに目覚ましいものがありますが、その一方で多くの懸念も抱えています。技術が進化することで、私たちの仕事が奪われたり、情報の信頼性が低下したりするリスクが高まっています。さらに、AIが生成するコンテンツが人間の創造性を脅かし、私たちの思考や感情に悪影響を及ぼす可能性もあります。結局のところ、便利さの裏には常に不安や恐れが潜んでいるのです。'}


### RunnableParallel の出力を Runnable の入力に連結する


In [108]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。2つの意見をまとめてください。"),
        ("human", "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}"),
    ]
)

In [109]:
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な見方では、生成AIの技術が進化することで、私たちの生活が便利で豊かになる可能性が広がり、クリエイティブな作業や問題解決のパートナーとしての役割を果たすことが期待されています。未来には新しい発見や革新が続くと信じられています。

一方で、悲観的な見方では、生成AIの進化には多くの懸念が伴い、仕事の喪失や情報の信頼性の低下といったリスクが高まることが指摘されています。また、AIが生成するコンテンツが人間の創造性を脅かし、思考や感情に悪影響を及ぼす可能性も懸念されています。このように、便利さの裏には不安や恐れが潜んでいるという意見もあります。

総じて、生成AIの進化は多くの可能性を秘めている一方で、慎重な対応が求められる課題も抱えていると言えるでしょう。


### RunnableParallel への自動変換


In [110]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

In [113]:
output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
# output = synthesize_chain.invoke({"topic": "財務省解体デモを実施することの是非について"})
# output = synthesize_chain.invoke({"topic": "Pay Money Too My PainのVocal Kの死亡経緯について"})
print(output)

Pay Money To My PainのVocal Kの訃報は、多くのファンにとって非常に悲しい出来事であり、彼の音楽やメッセージは今も多くの人々に影響を与えています。彼の歌は感情や思いを力強く表現し、ファンにとって大きな支えとなっていました。彼の遺したものは消えることなく、これからも多くの人に愛され続けるでしょう。

一方で、彼の死は音楽界における才能の喪失だけでなく、精神的な健康問題や孤独感の深刻な影響を再認識させるものであり、アーティストが抱える苦悩を浮き彫りにします。彼の死は、周囲の支えがあっても最終的には個人の闘いであることを思い知らされる警鐘でもあります。音楽は人々をつなげる力を持っていますが、孤独感や絶望感からは逃れられないこともあるのです。

このように、Vocal Kの訃報は彼の音楽の影響力を再確認させる一方で、音楽業界の裏側に潜む現実や精神的な苦しみについても考えさせられる出来事となっています。


### RunnableLambda との組み合わせ―itemgetter を使う例


In [114]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "生成AIの進化について"})
print(topic)

生成AIの進化について


In [115]:
from operator import itemgetter

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは客観的AIです。{topic}について2つの意見をまとめてください。",
        ),
        (
            "human",
            "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}",
        ),
    ]
)

synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic"),
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化に関する意見は、楽観的な見方と悲観的な見方の2つに分かれます。

**楽観的意見:** 生成AIの進化は、私たちの生活をより便利で豊かにする可能性を秘めています。技術の進歩により、クリエイティブな作業や問題解決の支援を行うAIが増え、私たちのアイデアや想像力を引き出す存在となっています。これからも新たな発見や革新が続くことで、未来は明るく、さまざまな素晴らしい体験が待っていると期待されています。

**悲観的意見:** 一方で、生成AIの進化には多くの懸念も伴います。技術の進化が進むことで、仕事の喪失や情報の信頼性の低下といったリスクが高まります。また、AIが生成するコンテンツが人間の創造性を脅かし、思考や感情に悪影響を及ぼす可能性も指摘されています。便利さの裏には常にリスクが存在し、技術の扱い方が未来の社会に大きな影響を与えることが懸念されています。

このように、生成AIの進化には明るい未来を期待する声と、リスクを警戒する声が共存しています。


## 5.4. RunnablePassthrough―入力をそのまま出力する


In [116]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [117]:
!pip install tavily-python==0.5.0

In [118]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [119]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

In [120]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("東京の今日の天気は？")
print(output)

東京の今日、2月8日（土）の天気は晴れで、最高気温は9℃、最低気温は-1℃です。降水確率は10%となっています。また、北西の風がやや強く吹く予想です。


### assign―RunnableParallel に値を追加する


In [122]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

# output = chain.invoke("東京の今日の天気は？")
output = chain.invoke("東京の3/17の天気は？")
pprint.pprint(output)

{'answer': '東京の3月17日（月）の天気は、冬型の気圧配置となり、おおむね晴れる見込みです。最高気温は16℃前後まで上昇し、寒暖差が大きくなるため、敏感な人は体調に注意が必要です。また、全国的に強風が吹く可能性があり、交通機関に影響が出るおそれもあります。',
 'context': [Document(metadata={'title': '3/17(月) 東京の天気頭痛予報 晴れるが寒暖差と花粉に注意 気圧の上昇に敏感な人は引き続き安静に | 頭痛ーる', 'source': 'https://zutool.jp/column/forecast/20250316-tokyo', 'score': 0.9381752, 'images': []}, page_content='3/17(月)の東京：冬型の気圧配置となり天気は回復しておおむね晴れる見込みです。最高気温は16℃前後まで上昇し、16日より8℃ほど高く、真冬から3月下旬の陽気となるでしょう。寒暖差が大きくなりますので敏感な人は体調に十分注意してく'),
             Document(metadata={'title': 'お天気キャスター解説 3月17日(月)の天気 - ウェザーニュース', 'source': 'https://weathernews.jp/s/topics/202503/170055/', 'score': 0.64236575, 'images': []}, page_content='きょう3月17日(月)の全国の天気をウェザーニュースキャスターの小林李衣奈がお伝えします。 一週間のスタートは、発達した低気圧の影響で全国的に強風が吹き荒れます。交通機関に影響するおそれがあるため最新の情報を'),
             Document(metadata={'title': '東京（東京）の天気 - Yahoo!天気・災害', 'source': 'https://weather.yahoo.co.jp/weather/jp/13/4410.html', 'score': 0.25258723, 'images': []}, page_content='パーソナル天気 現在位置： 天気・災害トップ > 関東・信越 > 東京都 > 東京（東京） 大雪に警戒

In [123]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [124]:
output = chain.invoke("東京の今日の天気は？")
print(output)

{'question': '東京の今日の天気は？', 'context': [Document(metadata={'title': '東京都の天気 - 日本気象協会 tenki.jp', 'source': 'https://tenki.jp/forecast/3/16/', 'score': 0.8875269, 'images': []}, page_content='東京都の天気 - 日本気象協会 tenki.jp tenki.jpトップ｜ サイトマップ｜ ヘルプ tenki.jp 検索 現在地 天気予報 天気予報 世界天気 2週間天気 長期予報 雨雲レーダー 積雪マップ PM2.5分布予測 黄砂情報 雷(予報) 道路気象 観測 雨雲レーダー(過去) アメダス 実況天気 過去天気 雷(実況) 防災情報 警報・注意報 地震情報 津波情報 火山情報 台風情報 知る防災 天気図 天気図 気象衛星 世界衛星 指数情報 洗濯 服装 お出かけ 星空 傘 紫外線 体感温度 洗車 睡眠 ヒートテック 水道凍結 うるおい 霜 風邪ひき 暖房 鍋もの 寒暖差 レジャー天気 山の天気 海の天気 空港 野球場 サッカー場 ゴルフ場 キャンプ場 競馬・競艇・競輪場 釣り お出かけスポット天気 季節特集 花粉飛散情報 桜開花情報 GWの天気 梅雨入り・明け 熱中症情報 紅葉見頃情報 ヒートショック予報 スキー積雪情報 初日の出 天気ニュース 気象予報士のポイント解説 季節・暮らしの話題 放送局のニュース 特集 雨雲レーダー 天気図 PM2.5分布予測 地震情報 気象予報士の解説 スキー積雪情報注目 トップ 関東・甲信地方 東京都 【お詫び】tenki.jpがご利用しづらい事象について 東京都の天気 08(土) 09(日) 10(月) 11(火) 12(水) 13(木) 14(金) 15(土) 16(日) 17(月) 18(火) 08日10:00発表02月08日(土) 千代田区 10/1 10% 新宿区 9/0 10% 世田谷区 9/0 10% 三鷹市 9/0 10% 八王子市 10/-1 10% 青梅市 9/-3 10% 大島町 9/3 10% 八丈町 8/3 50% 小笠原村 18/18 40% 練馬区 9/0 10% 台東区 10/1 10% 品川区 10/1 0% 町田市 10/

#### ＜補足：pick ＞


In [125]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | model | StrOutputParser())
    .pick(["context", "answer"])
)

In [126]:
output = chain.invoke("東京の今日の天気は？")
print(output)

{'context': [Document(metadata={'title': '東京都の天気 - 日本気象協会 tenki.jp', 'source': 'https://tenki.jp/forecast/3/16/', 'score': 0.8875269, 'images': []}, page_content='東京都の天気 - 日本気象協会 tenki.jp tenki.jpトップ｜ サイトマップ｜ ヘルプ tenki.jp 検索 現在地 天気予報 天気予報 世界天気 2週間天気 長期予報 雨雲レーダー 積雪マップ PM2.5分布予測 黄砂情報 雷(予報) 道路気象 観測 雨雲レーダー(過去) アメダス 実況天気 過去天気 雷(実況) 防災情報 警報・注意報 地震情報 津波情報 火山情報 台風情報 知る防災 天気図 天気図 気象衛星 世界衛星 指数情報 洗濯 服装 お出かけ 星空 傘 紫外線 体感温度 洗車 睡眠 ヒートテック 水道凍結 うるおい 霜 風邪ひき 暖房 鍋もの 寒暖差 レジャー天気 山の天気 海の天気 空港 野球場 サッカー場 ゴルフ場 キャンプ場 競馬・競艇・競輪場 釣り お出かけスポット天気 季節特集 花粉飛散情報 桜開花情報 GWの天気 梅雨入り・明け 熱中症情報 紅葉見頃情報 ヒートショック予報 スキー積雪情報 初日の出 天気ニュース 気象予報士のポイント解説 季節・暮らしの話題 放送局のニュース 特集 雨雲レーダー 天気図 PM2.5分布予測 地震情報 気象予報士の解説 スキー積雪情報注目 トップ 関東・甲信地方 東京都 【お詫び】tenki.jpがご利用しづらい事象について 東京都の天気 08(土) 09(日) 10(月) 11(火) 12(水) 13(木) 14(金) 15(土) 16(日) 17(月) 18(火) 08日10:00発表02月08日(土) 千代田区 10/1 10% 新宿区 9/0 10% 世田谷区 9/0 10% 三鷹市 9/0 10% 八王子市 10/-1 10% 青梅市 9/-3 10% 大島町 9/3 10% 八丈町 8/3 50% 小笠原村 18/18 40% 練馬区 9/0 10% 台東区 10/1 10% 品川区 10/1 0% 町田市 10/-1 10% 立川市 9/-1 10% 現在地の天気

### （コラム）astream_events


In [ ]:
# Google Colabでは次のコードの「async」の箇所に「Use of "async" not allowed outside of async function」と表示されますが、エラーなく実行できます

In [127]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    print(event, flush=True)

{'event': 'on_chain_start', 'data': {'input': '東京の今日の天気は？'}, 'name': 'RunnableSequence', 'tags': [], 'run_id': '7c0cebf4-92b5-497b-9960-f4935a5abecb', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnableParallel<context,question>', 'tags': ['seq:step:1'], 'run_id': '81594d5f-dad1-4021-8e2e-8792668a7908', 'metadata': {}, 'parent_ids': ['7c0cebf4-92b5-497b-9960-f4935a5abecb']}
{'event': 'on_retriever_start', 'data': {'input': {'query': '東京の今日の天気は？'}}, 'name': 'TavilySearchAPIRetriever', 'tags': ['map:key:context'], 'run_id': '4adb5434-0c1d-422f-8574-1808ada1bc4f', 'metadata': {'ls_retriever_name': 'tavilysearchapi'}, 'parent_ids': ['7c0cebf4-92b5-497b-9960-f4935a5abecb', '81594d5f-dad1-4021-8e2e-8792668a7908']}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnablePassthrough', 'tags': ['map:key:question'], 'run_id': '86696f0b-5374-498c-ab7a-b02d0c0db757', 'metadata': {}, 'parent_ids': ['7c0cebf4-92b5-497b-9960-f4935a5abecb', '81594d5f-dad1-4021-

In [128]:
async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    event_kind = event["event"]

    if event_kind == "on_retriever_end":
        print("=== 検索結果 ===")
        documents = event["data"]["output"]
        for document in documents:
            print(document)

    elif event_kind == "on_parser_start":
        print("=== 最終出力 ===")

    elif event_kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(chunk, end="", flush=True)

=== 検索結果 ===
page_content='東京都の天気 - 日本気象協会 tenki.jp tenki.jpトップ｜ サイトマップ｜ ヘルプ tenki.jp 検索 現在地 天気予報 天気予報 世界天気 2週間天気 長期予報 雨雲レーダー 積雪マップ PM2.5分布予測 黄砂情報 雷(予報) 道路気象 観測 雨雲レーダー(過去) アメダス 実況天気 過去天気 雷(実況) 防災情報 警報・注意報 地震情報 津波情報 火山情報 台風情報 知る防災 天気図 天気図 気象衛星 世界衛星 指数情報 洗濯 服装 お出かけ 星空 傘 紫外線 体感温度 洗車 睡眠 ヒートテック 水道凍結 うるおい 霜 風邪ひき 暖房 鍋もの 寒暖差 レジャー天気 山の天気 海の天気 空港 野球場 サッカー場 ゴルフ場 キャンプ場 競馬・競艇・競輪場 釣り お出かけスポット天気 季節特集 花粉飛散情報 桜開花情報 GWの天気 梅雨入り・明け 熱中症情報 紅葉見頃情報 ヒートショック予報 スキー積雪情報 初日の出 天気ニュース 気象予報士のポイント解説 季節・暮らしの話題 放送局のニュース 特集 雨雲レーダー 天気図 PM2.5分布予測 地震情報 気象予報士の解説 スキー積雪情報注目 トップ 関東・甲信地方 東京都 【お詫び】tenki.jpがご利用しづらい事象について 東京都の天気 08(土) 09(日) 10(月) 11(火) 12(水) 13(木) 14(金) 15(土) 16(日) 17(月) 18(火) 08日10:00発表02月08日(土) 千代田区 10/1 10% 新宿区 9/0 10% 世田谷区 9/0 10% 三鷹市 9/0 10% 八王子市 10/-1 10% 青梅市 9/-3 10% 大島町 9/3 10% 八丈町 8/3 50% 小笠原村 18/18 40% 練馬区 9/0 10% 台東区 10/1 10% 品川区 10/1 0% 町田市 10/-1 10% 立川市 9/-1 10% 現在地の天気 住所から検索 渋谷区 9 / 0 10% 豊島区 9 / 0 10% 江東区 10 / 1 10% 港区 10 / 1 10% 大田区 10 / 1 10% 府中市 9 / -1 10% 奥多摩町 6 / -4 10% その他の市区町村 関東地方は広く晴れて、沿岸部を中

### memo

betaからmigrateされたみたい。

https://python.langchain.com/docs/versions/v0_2/migrating_astream_events/

> **tip** <br>
Use the astream_events API to access custom data and intermediate outputs from LLM applications built entirely with LCEL.
>
> While this API is available for use with LangGraph as well, it is usually not necessary when working with LangGraph, as the stream and astream methods provide comprehensive streaming capabilities for LangGraph graphs.



### （コラム）Chat history と Memory


In [129]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [135]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def respond(session_id: str, human_message: str) -> str:
    chat_message_history = SQLChatMessageHistory(
        session_id=session_id, connection="sqlite:///sqlite.db"
    )

    ai_message = chain.invoke(
        {
            "chat_history": chat_message_history.get_messages(),
            "input": human_message,
        }
    )

    chat_message_history.add_user_message(human_message)
    chat_message_history.add_ai_message(ai_message)

    return ai_message

In [136]:
from uuid import uuid4

session_id = uuid4().hex

output1 = respond(
    session_id=session_id,
    human_message="こんにちは！私はジョンと言います！",
)
print(output1)

output2 = respond(
    session_id=session_id,
    human_message="私の名前が分かりますか？",
)
print(output2)

こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？
はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？
